# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/video_games_sales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import glob
import re

In [2]:
df = pd.read_csv('video_games_sales.csv')
df

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55787,55788,Indivisible,indivisible,Role-Playing,NaN,PC,505 Games,Lab Zero Games,NaN,NaN,...,NaN,NaN,NaN,NaN,2019.0,11th Apr 19,http://www.vgchartz.com/game/224682/indivisibl...,1,NaN,/games/boxart/full_6339426AmericaFrontccc.jpg
55788,55789,Lost Ember,lost-amber,Adventure,RP,PC,Mooneye Studios,Mooneye Studios,NaN,NaN,...,NaN,NaN,NaN,NaN,2019.0,11th Apr 19,http://www.vgchartz.com/games/game.php?id=2246...,1,NaN,/games/boxart/full_8893129AmericaFrontccc.jpg
55789,55790,Lost Ember,lost-amber,Adventure,RP,PS4,Mooneye Studios,Mooneye Studios,NaN,NaN,...,NaN,NaN,NaN,NaN,2019.0,11th Apr 19,http://www.vgchartz.com/game/224690/lost-ember...,1,NaN,/games/boxart/full_6514293AmericaFrontccc.jpg
55790,55791,Lost Ember,lost-amber,Adventure,RP,XOne,Mooneye Studios,Mooneye Studios,NaN,NaN,...,NaN,NaN,NaN,NaN,2019.0,11th Apr 19,http://www.vgchartz.com/game/224688/lost-ember...,1,NaN,/games/boxart/full_6799693AmericaFrontccc.jpg


1) Как критики относятся к спортивным играм?

In [3]:
# удалим строки с пустыми оценками критиков.
df1 = df[['Genre', 'Critic_Score', 'Platform']]
df1 = df1.dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6536 entries, 0 to 55653
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Genre         6536 non-null   object 
 1   Critic_Score  6536 non-null   float64
 2   Platform      6536 non-null   object 
dtypes: float64(1), object(2)
memory usage: 204.2+ KB


In [4]:
df_non_sport = df1[(df1['Genre'] != "Sports")]
df_non_sport.head()

,Genre,Critic_Score,Platform
1,Platform,10.0,NES
2,Racing,8.2,Wii
5,Role-Playing,9.4,GB
6,Platform,9.1,DS
8,Platform,8.6,Wii


In [5]:
# средний рейтинг критиков для игр не спорт
mean_score = df_non_sport['Critic_Score'].mean()
print(mean_score)

7.2045656634747255


In [6]:
df_sport = df1[(df1['Genre'] == "Sports")]
df_sport.head()

,Genre,Critic_Score,Platform
0,Sports,7.7,Wii
4,Sports,8.0,Wii
16,Sports,7.9,Wii
17,Sports,8.0,Wii
68,Sports,8.3,PS4


In [7]:
# средний рейтинг критиков для игр спорт
mean_score_sport = df_sport['Critic_Score'].mean()
print(mean_score_sport)

7.291424418604655


H0: средняя оценка спортивных >= средней оценки остальных игр

In [8]:
alpha = 0.05
result = st.ttest_1samp(df_sport['Critic_Score'], 7.204)

if (result.statistic > 0) & (result.pvalue / 2 < alpha):
    print('Отвергаем нулевую гипотезу, критикам больше нравятся спортивные игры')
else:
    print('Не отвергаем нулевую гипотезу, критики предпочитают не спортивные игры')
print(result)

Не отвергаем нулевую гипотезу, критики предпочитают не спортивные игры
Ttest_1sampResult(statistic=1.5590952323667622, pvalue=0.119434361948988)


statistic > 0, что свидетельствует тому, что у спорт игр средний рейтинг выше (средние говорят о том же). Но, т.к. pvalue > alpha, мы не можем утверждать о стат значимости отличий в рейтингах.

2) Критикам нравятся больше игры на PC или на PS4?

In [9]:
df_PC = df1[(df1['Platform'] == "PC")]
df_PS4 = df1[(df1['Platform'] == "PS4")]

In [10]:
# средний рейтинг критиков для игр на PC
mean_score_pc = df_PC['Critic_Score'].mean()
print(mean_score_pc)
# средний рейтинг критиков для игр на PS4
mean_score_ps4 = df_PS4['Critic_Score'].mean()
print(mean_score_ps4)

7.541849710982658
7.904587155963299


H0: средняя оценка на PC и PS4 одинаковая
H1: средняя оценка на PC и PS4 различается

In [11]:
result = st.ttest_ind(df_PC['Critic_Score'], df_PS4['Critic_Score'], equal_var=False)
print(result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся к играм на PC и PS4')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, критики по-разному относятся к играм на PC и PS4


statistic < 0, что свидетельствует о том, что критикам больше нравятся игры на PS4.
pvalue < alpha, значит различия значимы, и мы можем отвергнуть Н0 и предположить, что критикам больше нравятся игры на PS4.

3) Критикам больше нравятся стрелялки или стратегии?

In [12]:
df1['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Misc', 'Party',
       'Simulation', 'Action', 'Shooter', 'Action-Adventure', 'Fighting',
       'Strategy', 'Adventure', 'MMO', 'Music', 'Puzzle', 'Sandbox',
       'Board Game', 'Education', 'Visual Novel'], dtype=object)

In [13]:
df_Sho = df1[(df1['Genre'] == "Shooter")]
df_Str = df1[(df1['Genre'] == "Strategy")]

In [14]:
mean_score_sho = df_Sho['Critic_Score'].mean()
print(mean_score_pc)
mean_score_str = df_Str['Critic_Score'].mean()
print(mean_score_ps4)

7.541849710982658
7.904587155963299


H0: средняя оценка одинаковая
H1: средняя оценка различается

In [15]:
result = st.ttest_ind(df_Sho['Critic_Score'], df_Str['Critic_Score'], equal_var=False)
print(result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу


In [ ]:
т.к. pvalue > alpha, то мы не можем с удовлетворительной степенью достоверности утверждать, что оценки значимо различны.

## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  
6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  
7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;  
8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);  
9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [16]:
df2 = pd.read_csv('spam.csv')
df2

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [18]:
def class_message(param):
    if param == "ham":
        return 0
    elif param == "spam":
        return 1

In [19]:
df2['class_message'] = df2['Category'].apply(class_message)
df2.head()

,Category,Message,class_message
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


1) Привидите весь текст к нижнему регистру;

In [20]:
df2['Message'] = df2.Message.str.lower()
df2

,Category,Message,class_message
0,ham,"go until jurong point, crazy.. available only ...",0
1,ham,ok lar... joking wif u oni...,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor... u c already then say...,0
4,ham,"nah i don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...,1
5568,ham,will ü b going to esplanade fr home?,0
5569,ham,"pity, * was in mood for that. so...any other s...",0
5570,ham,the guy did some bitching but i acted like i'd...,0


2) Удалите мусорные символы;

In [21]:
df2['Message'].replace(regex=True, inplace=True, to_replace=r'[\W_]+', value=r' ')
df2

,Category,Message,class_message
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i don t think he goes to usf he lives arou...,0
...,...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...,1
5568,ham,will ü b going to esplanade fr home,0
5569,ham,pity was in mood for that so any other suggest...,0
5570,ham,the guy did some bitching but i acted like i d...,0


3) Удалите стоп-слова;

In [22]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df2['Message'] = [' '.join([w for w in x.split() if w not in stopwords_set]) for x in df2['Message'].tolist()]
df2

,Category,Message,class_message
0,ham,go jurong point crazy available bugis n great ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,u dun say early hor u c already say,0
4,ham,nah think goes usf lives around though,0
...,...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...,1
5568,ham,ü b going esplanade fr home,0
5569,ham,pity mood suggestions,0
5570,ham,guy bitching acted like interested buying some...,0


4) Привидите все слова к нормальной форме;

In [24]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
df2['lemma'] = [' '.join([wordnet_lemmatizer.lemmatize(w) for w in x.split()]) for x in df2['Message'].tolist()]
df2

,Category,Message,class_message,lemma
0,ham,go jurong point crazy available bugis n great ...,0,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni,0,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say,0,u dun say early hor u c already say
4,ham,nah think goes usf lives around though,0,nah think go usf life around though
...,...,...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...,1,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,ü b going esplanade fr home,0,ü b going esplanade fr home
5569,ham,pity mood suggestions,0,pity mood suggestion
5570,ham,guy bitching acted like interested buying some...,0,guy bitching acted like interested buying some...


In [ ]:
https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/

5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df2.lemma)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# чтобы разделить датасет на тестовый и тренировочный соединим матрицу со столбцом-признаком спам/неспам
df_concat = pd.concat([df2['class_message'], tfidf_matrix], axis=1, sort=False)
df_concat

,class_message,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;

In [28]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_concat, test_size=0.3, random_state=42)

In [29]:
y_train, y_test = df_train['class_message'], df_test['class_message']
del df_train['class_message']
del df_test['class_message']

7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;

In [30]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [31]:
lda = LinearDiscriminantAnalysis()

In [33]:
lda.fit(df_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [34]:
lda.predict(df_test)

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [35]:
result = pd.DataFrame([y_test.values, lda.predict(df_test)]).T

In [36]:
result

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
1667,0,0
1668,1,1
1669,0,0
1670,1,1


In [37]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(y_test, lda.predict(df_test))
#точность 96,7%

0.9677033492822966

8) Опишите результаты при помощи confusion_matrix;

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
confusion_matrix(y_test, lda.predict(df_test))

array([[1445,    3],
       [  51,  173]], dtype=int64)

1445 - определено верно сообщения не спам,   
3 - сообщение не спам, а модель предсказала спам (ошибка 1 рода),
51 - сообщение спам, а модель предсказала не спам (ошибка 2 рода),
173 - определено верно сообщения спам

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [ ]:
Объединим предсказания в один датасет .... понять как восстановить индексы

In [41]:
#извлечем индексы из тестовой выборки
test_mes = pd.DataFrame(y_test).reset_index()
test_mes = pd.DataFrame(test_mes['index'])
test_mes.head()

,index
0,3245
1,944
2,1044
3,2484
4,812


In [42]:
df_message_test = pd.DataFrame(lda.predict(df_test), columns=['predict'])
df_message_test.head()

,predict
0,0
1,0
2,0
3,0
4,0


In [43]:
df_message_test = test_mes.join(df_message_test)
df_message_test.head()

,index,predict
0,3245,0
1,944,0
2,1044,0
3,2484,0
4,812,0


In [44]:
#извлечем индексы из тренировочной выборки
train_mes = pd.DataFrame(y_train).reset_index()
train_mes = pd.DataFrame(train_mes['index'])
train_mes.head()

,index
0,708
1,4338
2,5029
3,4921
4,2592


In [45]:
df_message_train = pd.DataFrame(lda.predict(df_train), columns=['predict'])
df_message_train.head()

,predict
0,0
1,0
2,0
3,0
4,0


In [46]:
df_message_train = train_mes.join(df_message_train)
df_message_train.head()

,index,predict
0,708,0
1,4338,0
2,5029,0
3,4921,0
4,2592,0


In [47]:
df_message_pred = pd.concat([df_message_train, df_message_test], axis=0, sort=False)
df_message_pred.head()

,index,predict
0,708,0
1,4338,0
2,5029,0
3,4921,0
4,2592,0


In [48]:
#вернем индекс на место
df_message_pred = df_message_pred.set_index(df_message_pred.columns[0])
df_message_pred.head()

,predict
index,
708,0
4338,0
5029,0
4921,0
2592,0


добавим столбец с предсказанием в датасет

In [49]:
df_all = df2.join(df_message_pred)
df_all = df_all[['Category', 'Message', 'class_message', 'predict']]
df_all

,Category,Message,class_message,predict
0,ham,go jurong point crazy available bugis n great ...,0,0
1,ham,ok lar joking wif u oni,0,0
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1,1
3,ham,u dun say early hor u c already say,0,0
4,ham,nah think goes usf lives around though,0,0
...,...,...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...,1,1
5568,ham,ü b going esplanade fr home,0,0
5569,ham,pity mood suggestions,0,0
5570,ham,guy bitching acted like interested buying some...,0,0


In [50]:
df_all['false_predict'] = df_all['class_message'] - df_all['predict']
df_all.head()

,Category,Message,class_message,predict,false_predict
0,ham,go jurong point crazy available bugis n great ...,0,0,0
1,ham,ok lar joking wif u oni,0,0,0
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1,1,0
3,ham,u dun say early hor u c already say,0,0,0
4,ham,nah think goes usf lives around though,0,0,0


In [ ]:
#оставим только строки где false_predict = 0 (потому что не смог написать функцию ....)

In [52]:
len(filter_df_all)

54

In [51]:
filter_df_all = df_all[df_all['false_predict'] != 0]
filter_df_all

,Category,Message,class_message,predict,false_predict
68,spam,hear new divorce barbie comes ken stuff,1,0,1
191,spam,unique enough find 30th august www areyouuniqu...,1,0,1
227,spam,u meet ur dream partner soon ur career 2 flyng...,1,0,1
305,spam,sms ac blind date 4u rodds1 21 aberdeen united...,1,0,1
368,spam,discount code rp176781 stop messages reply sto...,1,0,1
517,spam,boltblue tones 150p reply poly mono eg poly3 1...,1,0,1
611,spam,22 days kick euro2004 u kept date latest news ...,1,0,1
690,spam,forwarded 448712404000 please call 08712404000...,1,0,1
731,spam,email alertfrom jeri stewartsize 2kbsubject lo...,1,0,1
751,spam,realize 40 years thousands old ladies running ...,1,0,1


In [ ]:
# подскажите, как правильно написать функцию, в которой в качетве параметров 2 столбца????

In [ ]:
def check_message(param1, param2):
    if param1 == param2:
        return 'True'
    else:
        return 'False'

In [ ]:
# не работает
df_all['false_predict'] = df_all.apply(check_message, axis=1, param1='class_message', param2='predict')

In [ ]:
# не работает
df_all['false_predict2'] = df_all[df_all['class_message'], df_all['predict']].apply(check_message)